# CSTR 
---

This notebook considers a CSTR (Continuously stirred tank reactor); a common operation in Chemical Engineering. The notebook covers two aspects:
1. Steady-state operation which is represented by a system of nonlinear equations
2. Transient operation which is represented by a system of ordinary differential equations

## System description

---

<img src="figures/CSTR_empty.png" alt="Schematic of a CSTR" width="200"/>

In the CSTR shown above the following reactions are taking place
\begin{align*}
 A+B\to C & \quad\text{with } r_{AB} = k_1 c_A c_B \\
 C+B\to D & \quad\text{with } r_{CB} = k_2 c_C c_B \\
 A \to E & \quad\text{with } r_{A} = k_3 c_A
\end{align*}

The kinetic data is known at two temperatures
$$
\begin{aligned}
& \text {Table with kinetic data at two temperatures}\\
&\begin{array}{c|c|c|c}
  & T=298\,K & T=315\,K & Units \\ 
\hline
k_1 & 2.1\times 10^{-2} & 3.6\times 10^{-2} & m^3\,mol^{-1}\,s^{-1} \\
k_2 & 1.5\times 10^{-2} & 4.5\times 10^{-2} & m^3\,mol^{-1}\,s^{-1} \\
k_3 & 1.2\times 10^{-4} & 2.6\times 10^{-4} & s^{-1}  
\end{array}
\end{aligned}
$$

The volume of the CSTR is $1\,$m$^3$ and the inlet and outlet flow rate are $0.001\,$m$^3\,$s$^{-1}$. The inlet contains the components A and B, with $c_{A,i}=0.5\,$M and a varying $\gamma=c_{B,i}/c_{A,i}$. 

## Part 1) Steady-state operation

### Task a) Derive the system of governing equations

---

The mass balance equations are derived in the same manner as in the lectures with the addition of an additional equation for the concentration of component E
\begin{align*}
0 &= F\left(c_{A,i} - c_A\right) - V (k_1 c_A c_B + k_3 c_A) \\
 0 &= F\left(c_{B,i} - c_B\right) - V (k_1 c_A c_B + k_2 c_C c_B)  \\
 0 &= -F c_C + V(k_1 c_A c_B - k_2 c_C c_B) \\
 0 &= -F c_D + V k_2 c_C c_B \\
 0 &= -F c_E + V k_3 c_A
\end{align*}
Each of the five nonlinear equations is written so that at a solution it equates to zero.

### Task b) Solve the nonlinear equation system

Assuming isothermal operation and no volume change, solve the nonlinear equation system and plot the conversion of A as a function of temperature and $\gamma$.

---

First, define a function that calculates and returns the left-hand side of the nonlinear algebraic equation system, i.e. $\mathbf{y}=[y[0], y[1], y[2], y[3], y[4]]$. At the solution $\mathbf{x}^*$ of the nonlinear algebraic equation system the array $\mathbf{y}$ is zero.

In [ ]:
import numpy as np
# CSTR right hand side function
def CSTR_rhs(x, parameters):
    y = np.zeros(5)
    # Assign parameterseters to local variables
    F = parameters[0]
    V = parameters[1]
    c_Ai = parameters[2]
    c_Bi = parameters[3]
    c_Ci = parameters[4] 
    c_Di = parameters[5]
    c_Ei = parameters[6]
    k1 = parameters[7]
    k2 = parameters[8]
    k3 = parameters[9]
    
    # Define the nonlinear equation system
    y[0] = F*(c_Ai - x[0]) - V * (k1*x[0]*x[1] + k3*x[0])
    y[1] = F*(c_Bi - x[1]) - V * (k1*x[0]*x[1] + k2*x[2]*x[1])
    y[2] = F*(c_Ci - x[2]) + V * (k1*x[0]*x[1] - k2*x[2]*x[1])
    y[3] = F*(c_Di - x[3]) + V * k2*x[2]*x[1]
    y[4] = F*(c_Ei - x[4]) + V * k3*x[0]
    
    return y

The following cell loops over two temperatures and values of $\gamma$ from $0$ to $4$. For each value of $\gamma$ the parameter array and the initial guess is set before solving the nonlinear algebraic equation system defined by the `CSTR_rhs` function. It then stores the value of $\gamma$ and the concentration of A in an array for later plotting. 

In [ ]:
# Solve CSTR 2nd order reactions
from scipy.optimize import fsolve

# Lists to store the concentration and gamma values
cA_all = []
Ts = [298, 315]

for T in Ts:
    cA = []
    gamma = []
    for ga in np.linspace(0, 4, 10):   # Loop over different gamma values
        # Parameter array: F, V, c_Ai, c_Bi, c_Ci, c_Di, c_Ei, k1, k2, k3

        ## Comment out one of the two parameter arrays
        if T == 298:
            parameters = [0.001, 1, 0.5, ga*0.5, 0, 0, 0, 2.1e-2, 1.5e-2, 1.2e-4]
        elif T == 315:
            parameters = [0.001, 1, 0.5, ga*0.5, 0, 0, 0, 3.6e-2, 4.5e-2, 2.6e-4]
        else:
            print("T needs to be either 298K or 315K")
            break

        # Initial guess
        x0 = [0.5/(1+2*ga), ga*0.5/(1+2*ga), 0, 0, 0]

        # Solve
        x = fsolve(CSTR_rhs, x0, args=(parameters))

        # Store gamma and c_A
        gamma.append(ga)
        cA.append(x[0])
    
    cA_all.append(cA)

The following cell plots the output for both temperatures against the values of $\gamma$.

In [ ]:
import matplotlib.pyplot as plt

for cA, T in zip(cA_all, Ts):
    cA = np.array(cA)

    plt.plot(gamma, 1 - cA/0.5)
    plt.xlabel(r'$\gamma$')
    plt.ylabel('Conversion of A')
    plt.title('Conversion of A for various concentrations at T={}K'.format(T))
    plt.show()

The figures show that the conversion of A is increased at the higher temperature due to the increase in reaction rates. 

### Task c)

What influence does the flow rate have on the conversion of A? Nondimensionalise the equation system to help in your analysis.

---

As the characteristic value for the concentrations we pick the input concentration of component $A$ which is $c_{A,i}$. Using the nondimensionalisations $c_A = c_{A,i}\tilde{c}_A$, $c_B = c_{A,i}\tilde{c}_B$, $c_C = c_{A,i}\tilde{c}_C$, $c_D = c_{A,i}\tilde{c}_D$ and $c_E = c_{A,i}\tilde{c}_E$ we get
\begin{align*}
0 &= \left(1 - \tilde{c}_A\right) - \frac{V}{F} (k_1 c_{A,i}\tilde{c}_A \tilde{c}_B + k_3 \tilde{c}_A) \\
 0 &= \left(\gamma - \tilde{c}_B\right) - \frac{V c_{A,i}}{F} (k_1 \tilde{c}_A \tilde{c}_B + k_2 \tilde{c}_C \tilde{c}_B)  \\
 0 &= - \tilde{c}_C + \frac{V c_{A,i}}{F}(k_1 \tilde{c}_A \tilde{c}_B - k_2 \tilde{c}_C \tilde{c}_B) \\
 0 &= - \tilde{c}_D + \frac{V k_2 c_{A,i}}{F} \tilde{c}_C \tilde{c}_B \\
 0 &= - \tilde{c}_E + \frac{V k_3}{F} \tilde{c}_A
\end{align*}


All the remaining parameters and in particular the two parameters influencing the conversion of A are inverse proportional to the flow rate $F$. Thus when we increase the flow rate the conversion of A is reduced while for a decrease of the flow rate the conversion is increased. The following plot shows the conversion for $F=0.0001\,$m$^3\,$s$^{-1}$. For values of $\gamma>2$ the conversion is almost complete.

*Remark:* You can test this by changing the flow rate and re-plotting the conversion of component $A$.

## Part 2) Analysis of transient CSTR operation

### Task a) Derive the transient system of equations

---



Following from Part 1a), the difference between the steady-state and transient CSTR operation is the time derivative of the number of moles of each component on the left-hand side so that the five differential equations are given by
\begin{align*}
 \frac{d}{dt} \left(V c_A\right) &= F\left(c_{A,i} - c_A\right) - V (k_1 c_A c_B + k_3 c_A) \\
 \frac{d}{dt} \left(V c_B\right) &= F\left(c_{B,i} - c_B\right) - V (k_1 c_A c_B + k_2 c_C c_B)  \\
 \frac{d}{dt} \left(V c_C\right) &= -F c_C + V(k_1 c_A c_B - k_2 c_C c_B) \\
 \frac{d}{dt} \left(V c_D\right) &= -F c_D + V k_2 c_C c_B \\
 \frac{d}{dt} \left(V c_E\right) &= -F c_E + V k_3 c_A
\end{align*}

### Task b) Solve and plot the differential equation system

---

Solve the system for the two temperatures and for $\gamma=2$.

You can reuse the right-hand side function from Part 1. You have to add the time parameter but no other modifications are needed for this system because none of the parameters are time-dependent.

In [ ]:
# Define the CSTR ODE system right-hand side
def CSTR_ODE_rhs(t, x, parameters):
    return CSTR_rhs(x, parameters)

In [ ]:
# Solve ODE system with the RK45 ODE solver
import numpy as np
from scipy.integrate import solve_ivp

# Temperature and gamma lists
Ts = [298, 315]
gamma = [2]

# Lists to store the concentration and gamma values
T_gamma = []
sols = []

for T in Ts:           # Loop over different temperature values
    for ga in gamma:   # Loop over different gamma values
        # Parameter array: F, V, c_Ai, c_Bi, c_Ci, c_Di, c_Ei, k1, k2, k3

        ## Comment out one of the two parameter arrays
        if T == 298:
            parameters = [0.001, 1, 0.5, ga*0.5, 0, 0, 0, 2.1e-2, 1.5e-2, 1.2e-4]
        elif T == 315:
            parameters = [0.001, 1, 0.5, ga*0.5, 0, 0, 0, 3.6e-2, 4.5e-2, 2.6e-4]
        else:
            print("T needs to be either 298K or 315K")
            break

        # Initial guess
        x0 = [0.5, ga*0.5, 0, 0, 0]

        # Solve
        sol = solve_ivp(CSTR_ODE_rhs, [0, 1000], x0, method='RK45', rtol=1e-3, args=[parameters])
        
        # Store T and gamma as well as the solution
        T_gamma.append({"T": T, "gamma": ga})
        sols.append(sol)

Plot the concentrations for the two temperatures.

In [ ]:
import matplotlib.pyplot as plt

# Define the plot
fig, ax = plt.subplots(1, 2, figsize=(15, 6))

for i in range(2):
    sol = sols[i]
    ax[i].plot(sol.t, sol.y[0], 'k-', label="A")
    ax[i].plot(sol.t, sol.y[1], 'r-.', label="B")
    ax[i].plot(sol.t, sol.y[2], 'g--', label="C")
    ax[i].plot(sol.t, sol.y[3], 'b:', label="D")
    ax[i].plot(sol.t, sol.y[4], 'c-.x', label="E")
    
    ax[i].set_xlabel("Nondimensional time [-]", fontsize=14)
    ax[i].set_ylabel("Nondimensional concentration [-]", fontsize=14)
    ax[i].set_title(r"Concentrations in the reactor for $\gamma$={} and T={}K".format(T_gamma[i]["gamma"], T_gamma[i]["T"]))
    
    # Setting the values for all axes.
    plt.setp(ax, xlim=(0, np.max(sol.t)))
    ax[i].legend()
plt.show()

### Task c) Investigate the solution time dependency on the stiffness of the system

---

For $T=315\,$K and $\gamma=2$, evaluate the time required to solve the system with different solvers for different reaction rates $k_2$: start with the given reaction rate and increase it by a factor of 10 after each simulation.

In [ ]:
# Solve ODE system with different ODE solvers
# Compare RK45 and Radau for systems with increasing stiffness
import numpy as np
from scipy.integrate import solve_ivp
import time

# Temperature and gamma lists
Ts = [315]
gamma = [2]

# Lists to store the parameter values and solutions
T_gamma = []
sols = []

for T in Ts:           # Loop over different temperature values
    for ga in gamma:   # Loop over different gamma values
        # Parameter array: F, V, c_Ai, c_Bi, c_Ci, c_Di, c_Ei, k1, k2, k3

        ## Comment out one of the two parameter arrays
        if T == 298:
            parameters = [0.001, 1, 0.5, ga*0.5, 0, 0, 0, 2.1e-2, 1.5e-2, 1.2e-4]
        elif T == 315:
            parameters = [0.001, 1, 0.5, ga*0.5, 0, 0, 0, 3.6e-2, 4.5e-2, 2.6e-4]
        else:
            print("T needs to be either 298K or 315K")
            break

        for i in range(6):      
            # Initial guess
            x0 = [0.5, ga*0.5, 0, 0, 0]
    
            # Solve
            start = time.time()
            print("Start time for ga={} is {:0.6f} seconds".format(ga, start))
            sol = solve_ivp(CSTR_ODE_rhs, [0, 1000], x0, method='Radau', rtol=1e-3, args=[parameters])
            time_Radau = time.time()-start
            
            # Add the solution with the Radau method to the list
            sols.append(sol)
            
            start = time.time()
            sol = solve_ivp(CSTR_ODE_rhs, [0, 1000], x0, method='RK45', rtol=1e-3, args=[parameters])
            time_RK45 = time.time()-start
    
            # Store parameters
            T_gamma.append({"T": T, "gamma": ga, "time_Radau": time_Radau, "time_RK45": time_RK45, "k2": parameters[8]})

            # Increase the reaction rate k2 for the next simulation
            parameters[8] = parameters[8] * 10

In [ ]:
# Define the plot
fig, ax = plt.subplots(1, 1, figsize=(15, 6))

k2s = []
tRadau = []
tRK45 = []

# Extract the data from the list of dictionaries
for par in T_gamma:
    k2s.append(par["k2"])
    tRadau.append(par["time_Radau"])
    tRK45.append(par["time_RK45"])
    
ax.plot(k2s, tRadau, 'k-x', label="Radau")
ax.plot(k2s, tRK45, 'g--+', label="RK45")
ax.set_xlabel(r"Reaction rate $k_2$ [m$^3\,$mol$^{-1}\,$s$^{-1}$]", fontsize=14)
ax.set_ylabel("Simulation time [s]", fontsize=14)
ax.set_yscale('log')
plt.xlim([0.0, np.max(k2s)])
ax.legend()
plt.show()